In [60]:
TEST_DIR = "test/"
import sys
stdout = sys.stdout
import csv
import pandas as pd
from components.Activation import Activation
from components.Layer import Layer

class FFNNModel:
    def __init__(self, layers):
        self.layers : list[Layer] = layers
        pass

    def setInitialInput(self, input: list[int]):
        if (len(self.layers) >= 1):
            self.layers[0].values = input

    def solve(self):
        for i in range(len(self.layers)-1):
            idxLayerInput = i
            idxLayerTarget = i+1
            targetValues = []
            temp=[]
            print("Layer: ", i)
            for nodeTarget in range(self.layers[idxLayerTarget].numOfNode):
                sigma = self.getSigma(idxLayerTarget, self.layers[idxLayerInput].values, nodeTarget)
                tempVal = Activation.active(sigma, self.layers[idxLayerTarget].activation)
                targetValues.append(tempVal)
                temp.append(sigma)
            print("Sigma: ", temp)
                
            self.layers[idxLayerTarget].values = targetValues
            print(targetValues)
        return self.layers[len(self.layers)-1].values

    
    def getSigma(self, idxLayer, input, node):
        temp = 0
        for idx, w in enumerate(self.layers[idxLayer].weights[node]):
            if (idx != 0):
                temp += w * input[idx-1]
            else:
                temp += w
        return temp

    @staticmethod
    def getModelFromFile (path):
        MODELS_DIR = "models/"
        layers: list[Layer] = []
        df = pd.read_json(MODELS_DIR + path)	
        for layer in df.model.layers :
            layer_ = Layer(layer['activation'], layer['numOfNode'], layer['weights'], layer['values'])
            layers.append(layer_)

        return FFNNModel(layers)

In [62]:
fnn = FFNNModel.getModelFromFile('model.json')

    
def isCorrect(result, target):
    for idx, res in enumerate(result):
        if (int(res) != int(target[idx])):
            return False

    return True

def isCorrect(result, target):
    for idx, res in enumerate(result):
        if (int(res) != int(target[idx])):
            return False
    return True

def solveModel(inp, target):
    result = []
    count = 0
    for idx, test in enumerate(inp):
        fnn.setInitialInput(test)
        res = fnn.solve()
        print("Input: ", test)
        print("Result:", res, end="  ")
        if (isCorrect(res, target[idx])):
            print("(Correct)")
            count += 1
        else:
            print("(Wrong)")
        result.append(res)
        
    print("Akurasi: ", float(count)/len(inp))

def readInputFromFile(filenamedir):
    with open(filenamedir, 'r') as f:
        data = [i.split(",") for i in f.read().split()]
    
    listInput = []
    listTarget = []
    temp =[]
    temp2 = []
    
    for i in range(len(data)):
        for j in range(len(data[i])):
            if i!=0:
                if j == 0 or j == 1:
                    temp.append(int(data[i][j]))
        listInput = slice_per(temp, len(data)-1)
        
    for i in range(len(data)):
        for j in range(len(data[i])):
            if i!=0:
                if j == 2:
                    temp2.append(int(data[i][j]))
        listTarget = slice_per(temp2, len(data)-1)
    
    return (listInput, listTarget)


def slice_per(source, step):
    return [source[i::step] for i in range(step)]

   
listInput, listTarget = readInputFromFile(TEST_DIR + "test.csv")
solveModel(listInput, listTarget)

Layer:  0
Sigma:  [10, 10]
[1, 1]
Layer:  1
Sigma:  [10]
[1]
Input:  [0, 1]
Result: [1]  (Wrong)
Layer:  0
Sigma:  [-10, 30]
[0, 1]
Layer:  1
Sigma:  [-10]
[0]
Input:  [0, 0]
Result: [0]  (Wrong)
Layer:  0
Sigma:  [10, 10]
[1, 1]
Layer:  1
Sigma:  [10]
[1]
Input:  [0, 1]
Result: [1]  (Correct)
Layer:  0
Sigma:  [30, -10]
[1, 0]
Layer:  1
Sigma:  [-10]
[0]
Input:  [1, 1]
Result: [0]  (Correct)
Akurasi:  0.5


# Visualization

In [15]:
# Model Visualization

from graphviz import Digraph
f = Digraph('Feed Forward Neural Network', filename='model 2.gv')
f.attr('node', shape='circle', fixedsize='true', width='0.9')

for i in range(len(fnn.layers)) :
    if i != 0:
        if i == 1:
            for j in range(len(fnn.layers[i].weights)) :
                for k in range(len(fnn.layers[i].weights[j])) :
                    if k == 0:
                        if j == 0:
                            f.edge(f'bx', f'h{i}_{k}', str(
                            fnn.layers[i].weights[j][k]))
                        else:
                            f.edge(f'bx', f'h{i}_{k+1}', str(
                                fnn.layers[i].weights[j][k]))
                    else :
                        if j == 0:
                            if k == 2:
                                f.edge(f'x{j}', f'h{i}_{k-1}', str(
                                fnn.layers[i].weights[j+1][k]))
                            else:
                                f.edge(f'x{j}', f'h{i}_{k-1}', str(
                                fnn.layers[i].weights[j][k]))
                        else:
                            if k == 1:
                                f.edge(f'x{j}', f'h{i}_{k-1}', str(
                                fnn.layers[i].weights[j-1][k]))
                            else:
                                f.edge(f'x{j}', f'h{i}_{k-1}', str(
                                fnn.layers[i].weights[j][k]))

        elif i ==2 :
            for j in range(len(fnn.layers[i].weights)):
                for k in range(len(fnn.layers[i].weights[j])):
                    if k == 0:
                        f.edge(f'bhx{i-1}', f'h{i}_{j}', str(
                            fnn.layers[i].weights[j][k]))
                    elif k == 1 :
                        f.edge(f'h{i-1}_{j}', f'h{i}_{0}', str(
                            fnn.layers[i].weights[j][k]))
                    else:
                        f.edge(f'h{i-1}_{j+1}', f'h{i}_{0}', str(
                            fnn.layers[i].weights[j][k]))
       
f.view()

'model 2.gv.pdf'